### Import python library

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import Dense, LSTM, Dropout
import tensorflow.keras.backend as K 

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go

from math import sqrt

### 1. Read data: augment_24group_1620.csv필요

In [2]:
PCT_lstm_train_X = pd.read_csv("ERA_lstm_final_train_X.csv")
PCT_lstm_train_y = pd.read_csv("ERA_lstm_final_train_y.csv")

PCT_lstm_test_X = pd.read_csv("ERA_lstm_final_test_X.csv")
PCT_lstm_test_y = pd.read_csv("ERA_lstm_final_test_y.csv")

In [3]:
team = list(PCT_lstm_train_X.T_ID.unique())
year = list(PCT_lstm_train_y.YEAR.unique())

In [4]:
team, year

(['LG', 'HH', 'NC', 'HT', 'SK', 'KT', 'WO', 'LT', 'SS', 'OB'],
 [2016, 2017, 2018, 2019])

#### 4) input shape로 변경 (row, timestep=2, feature)

ex) 
timestep = 2

* X_train_v 구성예시: [[1 ~ 24경기 데이터, 25 ~ 48경기 데이터], [49 ~ 72경기 데이터, 73 ~ 96경기 데이터] ]  
X_train_v.shape >> (2,2*x)             # x: 각 24group에 대한 변수 개수
* y_train_v 구성예시: 97 ~ 120 경기 승률

=> reshape

* X_train_v.shape >> (2,2,x)  # row, timestep, feature

### 2. 모델 구성(LSTM)
- optimizer: RMSprop -> lr(learning rate) 조절
- LSTM: 모델이 계속 동일한 결과값이 나올 때, input 뉴런 개수를 늘려야 한다는 글을 읽고 계속 input 노드 개수를 바꿔주면서 모델 생성중
- loss: MSE

- early_stop: patience를 크게하면 과적합 되는 경우가 있어서 최대한 작게 설정해둠
- batch_size: 모델이 계속 동일한 결과값이 나올 때, 데이터가 적어 batch size를 줄여보라는 글을 읽고 1로 설정해둠

In [5]:
model_dict = dict()
hist_dict = dict()
test_pred_df = pd.DataFrame([],columns = ['YEAR','T_ID','y','y_pred',"shift_ERA_1","shift_ERA_2",'rms','rms_avg'])

idx = 0
for y in year:
    tmp1 = PCT_lstm_train_X[PCT_lstm_train_X["YEAR"] == y]
    tmp2 = PCT_lstm_train_y[PCT_lstm_train_y["YEAR"] == y]
    tmp3 = PCT_lstm_test_X[PCT_lstm_test_X["YEAR"] == y]
    tmp4 = PCT_lstm_test_y[PCT_lstm_test_y["YEAR"] == y]
    for t in team:
        name = '{}{}'.format(y,t)
        print(name,"=======================================")
        
        X_train = tmp1[tmp1["T_ID"] == t].drop(["T_ID","YEAR"],axis = 1)
        y_train = tmp2[tmp2["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        X_test = tmp3[tmp3["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        y_test = tmp4[tmp4["T_ID"] == t].drop(["T_ID","YEAR"],axis=1)
        
        X_train_v = X_train.values
        y_train_v = y_train.values

        X_test_v = X_test.values
        y_test_v = y_test.values
        
        X_train_t = X_train_v.reshape(X_train_v.shape[0], 2,X_train_v.shape[1]//2)
        X_test_t = X_test_v.reshape(X_test_v.shape[0], 2,X_test_v.shape[1]//2)
        
        ## model
        K.clear_session() 

        model = Sequential()
        optimizer = Adam(lr=0.01)

        model.add(LSTM(100,input_shape = (2,X_train_v.shape[1]//2))) # (timestep, feature)
        model.add(Dense(1)) # output = 1
        model.compile(loss='mean_squared_error', optimizer=optimizer,metrics=['mae'])

        model.summary()
        
        
        early_stop = EarlyStopping(monitor='loss', mode = 'min',patience=2, verbose=1)

        hist1 = model.fit(X_train_t, y_train_v, epochs=100,
                  batch_size=1, verbose=1, callbacks=[early_stop])
        ##
        
        model_dict[name] = model
        hist_dict[name] = hist1
        
        y_pred = model.predict(X_test_t)
        rms = sqrt(mean_squared_error(y_test_v, y_pred))
        rms_avg = sqrt(mean_squared_error(y_test_v,[y_train.mean()[0]]))
        
        
        test_pred_df.loc[idx,:] = [y,t,y_test_v.reshape(-1)[0],y_pred.reshape(-1)[0],
                                  X_test.loc[X_test.index[0],["shift_ERA_1"]][0],
                                  X_test.loc[X_test.index[0],["shift_ERA_2"]][0], rms,rms_avg]

        idx += 1

test_pred_df[['y','y_pred',"shift_ERA_1","shift_ERA_2",'rms','rms_avg']] = test_pred_df[['y','y_pred',"shift_ERA_1","shift_ERA_2",'rms','rms_avg']].astype(float)

2016LG =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [==============================] - 3s 51ms/sample - loss: 1.8917 - mae: 1.0221
Epoch 2/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.9100 - mae: 0.8158
Epoch 3/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.7412 - mae: 0.7163
Epoch 4/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.6891 - mae: 0.7402
Epoch 5/100
49/49 [==============================] - 0s 4ms/sa

49/49 [==============================] - 2s 37ms/sample - loss: 1.9226 - mae: 0.8275
Epoch 2/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.4665 - mae: 0.6010
Epoch 3/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.4822 - mae: 0.5721
Epoch 4/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.4069 - mae: 0.5572
Epoch 5/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.3828 - mae: 0.5352
Epoch 6/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.3630 - mae: 0.4895
Epoch 7/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.3496 - mae: 0.5152
Epoch 8/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.4207 - mae: 0.5378
Epoch 9/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.4979 - mae: 0.5800
Epoch 00009: early stopping
2016WO =======================================
Model: "sequential"
____________________________________________________

49/49 [==============================] - 0s 3ms/sample - loss: 0.1195 - mae: 0.2671
Epoch 7/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.1231 - mae: 0.2653
Epoch 8/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.0876 - mae: 0.2307
Epoch 9/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.0790 - mae: 0.2160
Epoch 10/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.1222 - mae: 0.2890
Epoch 11/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.0927 - mae: 0.2396
Epoch 00011: early stopping
2017HH =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total par

49/49 [==============================] - 0s 3ms/sample - loss: 0.1978 - mae: 0.3522
Epoch 10/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.1168 - mae: 0.2708
Epoch 11/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0678 - mae: 0.2134
Epoch 12/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.1131 - mae: 0.2622
Epoch 13/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.0793 - mae: 0.2054
Epoch 00013: early stopping
2017LT =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
_________________________________

49/49 [==============================] - 3s 56ms/sample - loss: 1.5773 - mae: 0.7816
Epoch 2/100
49/49 [==============================] - 0s 5ms/sample - loss: 0.3201 - mae: 0.4749
Epoch 3/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.3046 - mae: 0.4640
Epoch 4/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.2478 - mae: 0.3977
Epoch 5/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.2406 - mae: 0.4102
Epoch 6/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.3306 - mae: 0.4587
Epoch 7/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.2541 - mae: 0.4076
Epoch 00007: early stopping
2018SK =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
______________________________________________

49/49 [==============================] - 2s 43ms/sample - loss: 1.4637 - mae: 0.7894
Epoch 2/100
49/49 [==============================] - 0s 4ms/sample - loss: 0.4962 - mae: 0.6047
Epoch 3/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.4535 - mae: 0.5411
Epoch 4/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.3821 - mae: 0.5144
Epoch 5/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.5123 - mae: 0.6038
Epoch 6/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.5132 - mae: 0.5992
Epoch 00006: early stopping
2019LG =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total para

49/49 [==============================] - 2s 32ms/sample - loss: 0.5667 - mae: 0.5201
Epoch 2/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.1682 - mae: 0.3287
Epoch 3/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.1995 - mae: 0.3669
Epoch 4/100
49/49 [==============================] - 0s 3ms/sample - loss: 0.1774 - mae: 0.3393
Epoch 00004: early stopping
2019LT =======================================
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               58400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 58,501
Trainable params: 58,501
Non-trainable params: 0
_________________________________________________________________
Train on 49 samples
Epoch 1/100
49/49 [===========================

In [7]:
test_pred_df.sort_values(by=["YEAR","T_ID"]).to_csv("ERA_t2.csv",index = False)

In [6]:
mean_squared_error(test_pred_df['y'],test_pred_df['y_pred'])

0.9296761015839238

In [9]:
test_pred_df.groupby(["YEAR"]).mean()

,y,y_pred,shift_ERA_1,shift_ERA_2,rms,rms_avg
YEAR,,,,,,
2016,4.972786,5.507199,5.462381,5.436733,0.777040,0.669363
2017,5.225316,5.325551,4.865418,5.241630,0.801455,0.738538
2018,5.383612,5.289339,5.780409,4.984139,0.638906,0.551212
2019,3.400941,4.152486,4.366891,4.444551,0.882577,1.036635


In [10]:
test_pred_df

,YEAR,T_ID,y,y_pred,shift_ERA_1,shift_ERA_2,rms,rms_avg
0,2016,LG,3.649770,5.645885,5.400000,5.551402,1.996116,1.736086
1,2016,HH,5.258114,6.307232,6.412500,5.010736,1.049118,0.147947
2,2016,NC,3.575342,5.700111,5.086342,5.057143,2.124769,1.669293
3,2016,HT,4.120827,5.078367,5.957746,4.779503,0.957541,1.158068
4,2016,SK,5.144882,4.747037,5.646226,5.391576,0.397844,0.147898
5,2016,KT,6.314516,6.744235,5.736422,7.130435,0.429719,0.622919
6,2016,WO,5.704839,5.434399,4.584112,5.039454,0.270440,0.622746
7,2016,LT,5.849294,5.430143,5.314286,5.869565,0.419150,0.074993
8,2016,SS,5.400000,5.307000,6.214286,5.070229,0.093000,0.297577
9,2016,OB,4.710280,4.677575,4.271889,5.467290,0.032705,0.216106


In [11]:
from sklearn.metrics import r2_score
tmp = test_pred_df.copy()
# tmp['half']= 0.5
r2_y_predict = r2_score(tmp['y'], tmp['y_pred'])
r2_y_predict

0.17232939349187404